# 0. Import

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from IPython.display import clear_output

import PIL

import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from helpers import*
from lenshelpers import*

from model.baseline import*
from model.densenet import*
from model.alexnet import*
from model.resnet18 import*
from model.vgg16 import*
from model.googLeNet import*
from model.squeezeNet import*

import warnings
warnings.filterwarnings('ignore') 


# 1. Building the data set

In [2]:
# Build the four classes 
config_repo_model = 'data/configFile/config_model'

size = 500
for i in np.arange(1,4):
    #model_name = config_repo_model + str(i) + '.yaml'
    res = Residual(size)
    res.build(i)

print('Data Generation Finished')

Data Generation Finished


In [3]:
metadata = pd.DataFrame()
for i in np.arange(1,4):
    [img, meta] = read_hdf5(i, path = "data/dataSet/")
    metadata = pd.concat([metadata,meta], ignore_index=True)
    if i == 1:
        final_array = img
    else:
         final_array = np.concatenate((final_array, img))
metadata ['ID'] = np.arange(0,final_array.shape[0])
data_set = CombineDataset(metadata,'ID','class',final_array)

print('Reading Data Finished')
data_train, data_test = train_test_split(data_set,train_size=0.9,random_state=42)

Reading Data Finished


In [4]:
batch_size_train = 50
batch_size_test = 10
max_epoch = 10


loader_train = DataLoader(data_train, batch_size = batch_size_train, shuffle = True, 
                          num_workers = 0, drop_last=True)

loader_test = DataLoader(data_test, batch_size = batch_size_test, shuffle = True, 
                         num_workers = 0, drop_last=True)

# 2. Baseline

## 2.1. Residual network

**Results** : 6 epochs - 0.993 accuracy

In [7]:
net = CNNNetBasic()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [8]:
test_acc = np.zeros(max_epoch)
# Optimal number of epochs : 6 epochs - 0.993 accuracy
for epoch in range(max_epoch):  # loop over the dataset multiple times

    train_net(loader_train, net, optimizer, criterion, epoch)
    mean_accuracy = test_net(loader_test,net)
    test_acc[epoch] = mean_accuracy

    print("epoch: {:.3f}, accuracy: {:.3f} ".format(epoch, mean_accuracy))
print('Finished Training')

NameError: name 'loader_train' is not defined

In [ ]:
f1_samples = 0
accuracy = 0
iteration = 0
with torch.no_grad():
    predictions = []
    targets = []
    for data in loader_test:
        images, meta_img, labels = data
        outputs = net(images)
        predictions.extend(outputs.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

        f1_samples+=result['samples/f1']
        accuracy+=result['samples/recall']
        iteration+=1


mean_f1 = f1_samples/(iteration+1)
mean_accuracy =accuracy/(iteration+1)

print("accuracy: {:.3f} "
        "samples f1: {:.3f}".format(mean_accuracy,
                                    mean_f1))

## 2.2. Metadata network
**Results** : 4 epochs - 0.865 accuracy 

In [ ]:
net = TabularNetBasic()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_acc_meta = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    train_net(loader_train, net, optimizer, criterion, epoch)
    mean_accuracy = test_net(loader_test,net)
    test_acc_meta[epoch] = mean_accuracy

    print("epoch: {:.3f}, accuracy: {:.3f} ".format(epoch, mean_accuracy))
print('Finished Training')

## 2.3. Tabular & Residual network
**Results** : 8 epochs -  0.995 accuracy

In [ ]:
net = TabularCNNNetBasic()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_acc_metaxres = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    train_net(loader_train, net, optimizer, criterion, epoch)
    mean_accuracy = test_net(loader_test,net)
    test_acc_metaxres[epoch] = mean_accuracy

    print("epoch: {:.3f}, accuracy: {:.3f} ".format(epoch, mean_accuracy))
print('Finished Training')

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")

sns.lineplot(x=np.arange(0,max_epoch), y=test_acc, label = 'Residual')
sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_meta, label = 'Metadata')
sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_metaxres, label = 'Metadata & Residual')
# change legend texts
plt.xlabel('Epochs [-]')
plt.ylabel('Accuracy [-]')

# 3. Building different model

## 3.1. AlexNet - Residual maps
### 3.1.2 Padding
**Results** :  epochs -  accuracy 

In [16]:
net = AlexNetResidual()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [21]:
test_acc_pad = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        m = nn.ZeroPad2d(80)
        inputs = m(inputs)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    accuracy = 0
    iteration = 0
    with torch.no_grad():
        predictions = []
        targets = []
        for data in loader_test:
            images, meta_img, labels = data
            m = nn.ZeroPad2d(80)
            images = m(images)
            outputs = net(images)
            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

            accuracy+=result['samples/recall']
            iteration+=1


    test_acc_pad[epoch] =accuracy/(iteration+1)

    print("epoch:{:3d} \n test:\n "
            "accuracy: {:.3f} ".format(epoch, accuracy/(iteration+1)))
    
print('Finished Training')

KeyboardInterrupt: 

### 3.1.2. Interpolation

**Results** :  epochs -  accuracy 

In [ ]:
net = AlexNetResidual()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_acc_int = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = F.interpolate(inputs, size=(224, 224), mode='bicubic', align_corners=False)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    accuracy = 0
    iteration = 0
    with torch.no_grad():
        predictions = []
        targets = []
        for data in loader_test:
            images, meta_img, labels = data
            images = F.interpolate(images, size=(224, 224), mode='bicubic', align_corners=False)
            outputs = net(images)
            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

            accuracy+=result['samples/recall']
            iteration+=1


    test_acc_int[epoch] =accuracy/(iteration+1)

    print("epoch:{:3d} \n test:\n "
            "accuracy: {:.3f} ".format(epoch, accuracy/(iteration+1)))
    
print('Finished Training')

In [ ]:
sns.set_theme(style="whitegrid")

sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_pad, label = 'Padding')
sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_int, label = 'Interpolation')
# change legend texts
plt.xlabel('Epochs [-]')
plt.ylabel('Accuracy [-]')

## 3.2. Resnet18 - Residual maps
### 3.2.1. Padding
**Results** :  epochs -  accuracy 

In [ ]:
net = resnet18maps(1,3)
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_acc_pad = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        m = nn.ZeroPad2d(80)
        inputs = m(inputs)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    accuracy = 0
    iteration = 0
    with torch.no_grad():
        predictions = []
        targets = []
        for data in loader_test:
            images, meta_img, labels = data
            m = nn.ZeroPad2d(80)
            images = m(images)
            outputs = net(images)
            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

            accuracy+=result['samples/recall']
            iteration+=1


    test_acc_pad[epoch] =accuracy/(iteration+1)

    print("epoch:{:3d} \n test:\n "
            "accuracy: {:.3f} ".format(epoch, accuracy/(iteration+1)))
    
print('Finished Training')

### 3.2.2. Interpolation
**Results** :  epochs -  accuracy 

In [ ]:
net = resnet18maps(1,3)
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_acc_int = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = F.interpolate(inputs, size=(224, 224), mode='bicubic', align_corners=False)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    accuracy = 0
    iteration = 0
    with torch.no_grad():
        predictions = []
        targets = []
        for data in loader_test:
            images, meta_img, labels = data
            images = F.interpolate(images, size=(224, 224), mode='bicubic', align_corners=False)
            outputs = net(images)
            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

            accuracy+=result['samples/recall']
            iteration+=1


    test_acc_int[epoch] =accuracy/(iteration+1)

    print("epoch:{:3d} \n test:\n "
            "accuracy: {:.3f} ".format(epoch, accuracy/(iteration+1)))
    
print('Finished Training')

In [ ]:
sns.set_theme(style="whitegrid")

sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_pad, label = 'Padding')
sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_int, label = 'Interpolation')
# change legend texts
plt.xlabel('Epochs [-]')
plt.ylabel('Accuracy [-]')

## 3.3. VGG16 - Residual maps
### 3.3.1. Padding
**Results** :  epochs -  accuracy 

In [ ]:
net = VGG16Residual()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_acc_pad = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        m = nn.ZeroPad2d(80)
        inputs = m(inputs)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    accuracy = 0
    iteration = 0
    with torch.no_grad():
        predictions = []
        targets = []
        for data in loader_test:
            images, meta_img, labels = data
            m = nn.ZeroPad2d(80)
            images = m(images)
            outputs = net(images)
            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

            accuracy+=result['samples/recall']
            iteration+=1


    test_acc_pad[epoch] =accuracy/(iteration+1)

    print("epoch:{:3d} \n test:\n "
            "accuracy: {:.3f} ".format(epoch, accuracy/(iteration+1)))
    
print('Finished Training')

### 3.3.2 Interpolation
**Results** :  epochs -  accuracy 

In [ ]:
net = VGG16Residual()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_acc_int = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = F.interpolate(inputs, size=(224, 224), mode='bicubic', align_corners=False)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    accuracy = 0
    iteration = 0
    with torch.no_grad():
        predictions = []
        targets = []
        for data in loader_test:
            images, meta_img, labels = data
            images = F.interpolate(images, size=(224, 224), mode='bicubic', align_corners=False)
            outputs = net(images)
            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

            accuracy+=result['samples/recall']
            iteration+=1


    test_acc_int[epoch] =accuracy/(iteration+1)

    print("epoch:{:3d} \n test:\n "
            "accuracy: {:.3f} ".format(epoch, accuracy/(iteration+1)))
    
print('Finished Training')

In [ ]:
sns.set_theme(style="whitegrid")

sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_pad, label = 'Padding')
sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_int, label = 'Interpolation')
# change legend texts
plt.xlabel('Epochs [-]')
plt.ylabel('Accuracy [-]')

## 3.4. Dense Net 161 - Residual maps
### 3.4.1 Padding
**Results** :  epochs -  accuracy 

In [ ]:
net = densenet161()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_acc_pad = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        m = nn.ZeroPad2d(80)
        inputs = m(inputs)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    accuracy = 0
    iteration = 0
    with torch.no_grad():
        predictions = []
        targets = []
        for data in loader_test:
            images, meta_img, labels = data
            m = nn.ZeroPad2d(80)
            images = m(images)
            outputs = net(images)
            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

            accuracy+=result['samples/recall']
            iteration+=1


    test_acc_pad[epoch] = accuracy/(iteration+1)

    print("epoch:{:3d} \n test:\n "
            "accuracy: {:.3f} ".format(epoch, accuracy/(iteration+1)))
    
print('Finished Training')

### 3.4.2 Interpolation
**Results** :  epochs -  accuracy 

In [ ]:
net = densenet161()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_acc_int = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = F.interpolate(inputs, size=(224, 224), mode='bicubic', align_corners=False)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    accuracy = 0
    iteration = 0
    with torch.no_grad():
        predictions = []
        targets = []
        for data in loader_test:
            images, meta_img, labels = data
            images = F.interpolate(images, size=(224, 224), mode='bicubic', align_corners=False)
            outputs = net(images)
            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

            accuracy+=result['samples/recall']
            iteration+=1


    test_acc_int[epoch] =accuracy/(iteration+1)

    print("epoch:{:3d} \n test:\n "
            "accuracy: {:.3f} ".format(epoch, accuracy/(iteration+1)))
    
print('Finished Training')

In [ ]:
sns.set_theme(style="whitegrid")

sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_pad, label = 'Padding')
sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_int, label = 'Interpolation')
# change legend texts
plt.xlabel('Epochs [-]')
plt.ylabel('Accuracy [-]')

## 3.5. Google Net - Residual maps
### 3.5.1 Padding
**Results** :  epochs -  accuracy 

In [5]:
net = GoogLeNet(googlenet(True, True, None,224, dropout_rate=0.2, num_classes=3))
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [8]:
test_acc_pad = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        m = nn.ZeroPad2d(80)
        inputs = m(inputs)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, aux1, aux2 = net(inputs)
        loss = criterion((outputs+aux1+aux2)/3, labels)
        loss.backward()
        optimizer.step()


    accuracy = 0
    iteration = 0
    with torch.no_grad():
        predictions = []
        targets = []
        for data in loader_test:
            images, meta_img, labels = data
            m = nn.ZeroPad2d(80)
            images = m(images)
            outputs,aux1,aux2 = net(images)
            outputs = (outputs+aux1+aux2)/3
            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

            accuracy+=result['samples/recall']
            iteration+=1


    test_acc_pad[epoch] =accuracy/(iteration+1)

    print("epoch:{:3d} \n test:\n "
            "accuracy: {:.3f} ".format(epoch, accuracy/(iteration+1)))
    
print('Finished Training')

KeyboardInterrupt: 

### 3.5.2 Interpolation
**Results** :  epochs -  accuracy 

In [ ]:
net = GoogLeNet(googlenet(True, True, None,224, dropout_rate=0.2, num_classes=3))
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_acc_int = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = F.interpolate(inputs, size=(224, 224), mode='bicubic', align_corners=False)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, aux1, aux2 = net(inputs)
        loss = criterion((outputs+aux1+aux2)/3, labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    accuracy = 0
    iteration = 0
    with torch.no_grad():
        predictions = []
        targets = []
        for data in loader_test:
            images, meta_img, labels = data
            images = F.interpolate(images, size=(224, 224), mode='bicubic', align_corners=False)
            outputs,aux1,aux2 = net(images)
            outputs = (outputs+aux1+aux2)/3
            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

            accuracy+=result['samples/recall']
            iteration+=1


    test_acc_int[epoch] =accuracy/(iteration+1)

    print("epoch:{:3d} \n test:\n "
            "accuracy: {:.3f} ".format(epoch, accuracy/(iteration+1)))
    
print('Finished Training')

In [ ]:
sns.set_theme(style="whitegrid")

sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_pad, label = 'Padding')
sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_int, label = 'Interpolation')
# change legend texts
plt.xlabel('Epochs [-]')
plt.ylabel('Accuracy [-]')

## 3.6. Squeeze Net 1_1  - Residual maps
### 3.6.1 Padding
**Results** :  epochs -  accuracy 

In [4]:
net = squeezenet1_1()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_acc_pad = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        m = nn.ZeroPad2d(80)
        inputs = m(inputs)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    accuracy = 0
    iteration = 0
    with torch.no_grad():
        predictions = []
        targets = []
        for data in loader_test:
            images, meta_img, labels = data
            m = nn.ZeroPad2d(80)
            images = m(images)
            outputs = net(images)
            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

            accuracy+=result['samples/recall']
            iteration+=1


    test_acc_pad[epoch] =accuracy/(iteration+1)

    print("epoch:{:3d} \n test:\n "
            "accuracy: {:.3f} ".format(epoch, accuracy/(iteration+1)))
    
print('Finished Training')

### 3.6.2 Interpolation
**Results** :  epochs -  accuracy 

In [ ]:
net = squeezenet1_1()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_acc_int = np.zeros(max_epoch)
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, meta_inputs,labels = data
        inputs = F.interpolate(inputs, size=(224, 224), mode='bicubic', align_corners=False)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    accuracy = 0
    iteration = 0
    with torch.no_grad():
        predictions = []
        targets = []
        for data in loader_test:
            images, meta_img, labels = data
            images = F.interpolate(images, size=(224, 224), mode='bicubic', align_corners=False)
            outputs = net(images)
            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            result = calculate_metrics(np.round(np.array(predictions)), np.array(targets))

            accuracy+=result['samples/recall']
            iteration+=1


    test_acc_int[epoch] =accuracy/(iteration+1)

    print("epoch:{:3d} \n test:\n "
            "accuracy: {:.3f} ".format(epoch, accuracy/(iteration+1)))
    
print('Finished Training')

In [ ]:
sns.set_theme(style="whitegrid")

sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_pad, label = 'Padding')
sns.lineplot(x=np.arange(0,max_epoch), y=test_acc_int, label = 'Interpolation')
# change legend texts
plt.xlabel('Epochs [-]')
plt.ylabel('Accuracy [-]')